In [6]:
from scipy import stats
import numpy as np

rng = np.random.default_rng()

In [7]:
import sys, os
cwd = os.getcwd()
sys.path.append(cwd + '/../.') 

import numpy as np
from sklearn.metrics import f1_score

In [8]:

al_steps=100
sample_size_per_step=1
save_and_repeat=5

name_postfix = ""

In [126]:
global rnd_micro 
rnd_micro = []
global rnd_macro 
rnd_macro= []

def print_f1_p(dataset, encoding, query_strategy, f1_micros_rnd=None,  f1_macros_rnd=None, index=-1):
    
    index_postfix = ''
    if index != -1:
        index_postfix = f'_{index}'
    
    name_postfix = ''
    if query_strategy == 'rnd':
        name_postfix = "_rnd"
        query_strategy = 'unc'
    load_dir = f'./res4/{dataset}_{encoding}_{str(query_strategy)}_{name}{name_postfix}_{sample_size_per_step}_{al_steps}_{save_and_repeat}{index_postfix}'
    load_dir2 = f'../eval_proxy/res/{dataset}_{encoding}_{str(query_strategy)}_{name}{name_postfix}_{sample_size_per_step}_{al_steps}_{save_and_repeat}'

    f1_micros = [] 
    f1_macros = []
    
    # BERT 
    for i in range(5):
        y_pred = np.load(f'{load_dir}/y_pred_{i}.npy')
        y_true = np.load(f'{load_dir}/y_true_{i}.npy')
        f1_micros.append(f1_score(y_true, y_pred, average='micro'))
        f1_macros.append(f1_score(y_true, y_pred, average='macro'))
        
    f1_micros = np.array(f1_micros)
    f1_macros = np.array(f1_macros)
    
    # Proxy
    f1_mic_list = np.load('%s/f1_micro.npy' % load_dir2)
    f1_mac_list = np.load('%s/f1_macro.npy' % load_dir2)
    f1_mic_list = np.array(f1_mic_list).mean(axis=0)
    f1_mac_list = np.array(f1_mac_list).mean(axis=0)
    
    
    print(encoding, '|' , round(f1_micros.mean(), 4), round(f1_micros.std(), 4), '|' ,round(f1_macros.mean(), 4), round(f1_macros.std(), 4))

    if name_postfix == "_rnd":
        f1_micros_rnd = f1_micros 
        f1_macros_rnd = f1_macros
    
    
    #print('##', f1_mic_list[:-200])
    #print('##', f1_micros.shape)
    
    mode = -1
    if index == '-1':
        pass
    if index == '-3':
        #f1_mic_list = f1_mic_list[:300]
        #f1_mac_list = f1_mac_list[:300]
        mode = -200
    
    
    print(stats.ttest_ind(f1_micros_rnd, f1_micros ,equal_var=True), stats.ttest_ind(f1_macros_rnd, f1_macros ,equal_var=True))
    print('BERT ', f1_micros.mean() , 'Proxy ', f1_mic_list[mode])
    s = (f1_micros.mean() - f1_mic_list[mode] )/f1_mic_list[mode]*100
    s2 = (f1_macros.mean()- f1_mac_list[mode])/f1_mac_list[mode]*100
    print('Diff',  round(s, 2), round(s2, 2))
    print('BERT(RND) -> BERT(Proxy)',  round((f1_micros.mean()  - f1_micros_rnd.mean() )/f1_micros_rnd.mean() *100, 2), round((f1_macros.mean() - f1_macros_rnd.mean() )/f1_macros_rnd.mean() *100, 2))
    
    if name_postfix == "_rnd":
        pass
    else:
        mean_lis.append(f1_macros.mean())
    
    if name_postfix == "_rnd":
        return f1_micros,  f1_macros
    
    print()

In [135]:
name='final'
data_set = 'reuters'
encoding = 'elmo'
index = '-3'


f1_micros,  f1_macros = print_f1_p(data_set, encoding, 'rnd', index=index)
print('___')

mean_lis = [] 

strategy = ['unc', 'unc_density', 'isli', 'isli_density', 'isls', 'isls_density']
for i in strategy:
    print(i, '', end='')
    print_f1_p(data_set, encoding, i, f1_micros_rnd=f1_micros,  f1_macros_rnd=f1_macros, index=index)

print('__')
print(np.array(mean_lis))
print(round(np.array(mean_lis).mean()*100, 2))

elmo | 0.8885 0.0196 | 0.6717 0.0766
Ttest_indResult(statistic=0.0, pvalue=1.0) Ttest_indResult(statistic=0.0, pvalue=1.0)
BERT  0.8884931506849314 Proxy  0.90187
Diff -1.48 -16.02
BERT(RND) -> BERT(Proxy) 0.0 0.0
___
unc elmo | 0.9143 0.0167 | 0.7925 0.0321
Ttest_indResult(statistic=-2.0012008614048127, pvalue=0.08036658194235903) Ttest_indResult(statistic=-2.9095868133362344, pvalue=0.01960151063857148)
BERT  0.9142922374429224 Proxy  0.935528
Diff -2.27 -9.24
BERT(RND) -> BERT(Proxy) 2.9 17.98

unc_density elmo | 0.9233 0.0112 | 0.7971 0.0337
Ttest_indResult(statistic=-3.082621128901817, pvalue=0.015056971166505116) Ttest_indResult(statistic=-2.9961257174393916, pvalue=0.0171727963394611)
BERT  0.9232876712328766 Proxy  0.9348380000000001
Diff -1.24 -9.14
BERT(RND) -> BERT(Proxy) 3.92 18.67

isli elmo | 0.9059 0.009 | 0.7237 0.0185
Ttest_indResult(statistic=-1.610090735913048, pvalue=0.14604404580025926) Ttest_indResult(statistic=-1.317900624908024, pvalue=0.22402239291679504)
BERT 

In [137]:
name='final'
data_set = 'app_store'
encoding = 'elmo'


f1_micros,  f1_macros = print_f1_p(data_set, encoding, 'rnd')
print('___')

mean_lis = [] 

strategy = ['unc', 'unc_density', 'isli', 'isli_density', 'isls', 'isls_density']
for i in strategy:
    print(i, '', end='')
    print_f1_p(data_set, encoding, i, f1_micros_rnd=f1_micros,  f1_macros_rnd=f1_macros)

print('__')
print(np.array(mean_lis))
print(round(np.array(mean_lis).mean()*100, 2))

elmo | 0.8476 0.0066 | 0.8048 0.0107
Ttest_indResult(statistic=0.0, pvalue=1.0) Ttest_indResult(statistic=0.0, pvalue=1.0)
BERT  0.8476220275344181 Proxy  0.8161459999999998
Diff 3.86 5.15
BERT(RND) -> BERT(Proxy) 0.0 0.0
___
unc elmo | 0.8623 0.0044 | 0.8241 0.0041
Ttest_indResult(statistic=-3.69495028410094, pvalue=0.006086672297706773) Ttest_indResult(statistic=-3.382259012291987, pvalue=0.009609450853359241)
BERT  0.8623279098873592 Proxy  0.8319139999999999
Diff 3.66 4.63
BERT(RND) -> BERT(Proxy) 1.73 2.4

unc_density elmo | 0.8608 0.0053 | 0.8211 0.0043
Ttest_indResult(statistic=-3.1056288696349275, pvalue=0.014541348379922327) Ttest_indResult(statistic=-2.8249420236388048, pvalue=0.02232335466351084)
BERT  0.8608260325406759 Proxy  0.8303499999999999
Diff 3.67 4.49
BERT(RND) -> BERT(Proxy) 1.56 2.02

isli elmo | 0.8631 0.0058 | 0.8266 0.0073
Ttest_indResult(statistic=-3.5155208387675208, pvalue=0.007897497229823129) Ttest_indResult(statistic=-3.378081395512822, pvalue=0.00966909

In [ ]:
\multirow{-9}{*}{\begin{sideways}$0\%$ human noise\end{sideways}}\\

In [19]:
name='final'
data_set = 'reuters'
encoding = 'elmo'
index = '-3'


f1_micros,  f1_macros = print_f1_p(data_set, encoding, 'rnd')
print('___')

mean_lis = [] 

strategy = ['unc', 'unc_density', 'isli', 'isli_density', 'isls', 'isls_density']
for i in strategy:
    print(i, '', end='')
    print_f1_p(data_set, encoding, i, f1_micros_rnd=f1_micros,  f1_macros_rnd=f1_macros, index=index)

print('__')
print(np.array(mean_lis))
print(round(np.array(mean_lis).mean(), 2))

elmo | 0.925 0.0091 | 0.7575 0.0314
Ttest_indResult(statistic=0.0, pvalue=1.0) Ttest_indResult(statistic=0.0, pvalue=1.0)
Diff 1.28 -8.52
BERT(RND) -> BERT(Proxy) 0.0 0.0
___
unc 

FileNotFoundError: [Errno 2] No such file or directory: '../eval_proxy/res/reuters_elmo_unc_final_1_100_5_-3/f1_micro.npy'

In [13]:
def ttest(root_dir):
    f1_mic_list = np.load('%s/f1_micro.npy' % root_dir)
    f1_mac_list = np.load('%s/f1_macro.npy' % root_dir)
    print(-1)

In [14]:
ttest(dirs[0])

FileNotFoundError: [Errno 2] No such file or directory: './res/app_store_sbert_unc_test_1_100_5/f1_micro.npy'

In [6]:
a = [0.5443,
0.2629,
0.2629,
0.4615,
0.2435,
0.2398,
0.2098,
0.206,
0.2036,
]

b = [0.4259,
0.2059,
0.2076,
0.3174,
0.146,
0.1464,
0.1592,
0.1542,
0.1513]


In [8]:
rvs1 = stats.norm.rvs(loc=5, scale=10, size=500, random_state=rng)

rvs2 = stats.norm.rvs(loc=5, scale=10, size=500, random_state=rng)

stats.ttest_ind(a, b,equal_var=True)

Ttest_indResult(statistic=1.5321758209957235, pvalue=0.14501165733864146)